Scraping data off of whosampled.com for project with Dr. Kenton Rambsy

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [4]:
url = "https://www.whosampled.com/Eminem/"
page = urlopen(url)
soup = BeautifulSoup(page, "html.parser")

In [5]:
tracks = soup.find_all('article', class_ = 'trackItem')
print(type(tracks))
print(len(tracks))

<class 'bs4.element.ResultSet'>
10


In [141]:
for track in tracks:
    # Get the details of the current song on the list
    trackName = track.h3.a.text
    trackRef = "https://www.whosampled.com" + track.h3.a['href']
    trackReleaseYear = track.h3.find('span', class_="trackYear").text
    print(trackName)
    
    # Get the details of the sampled songs
    track_connections = track.find_all('div', class_ = "track-connection")
    for track_connection in track_connections:
        sample = track_connection.span.text.strip()
        href = "https://www.whosampled.com" + track_connection.ul.li.a['href']
        print("This track", sample, " : ", href)
        pageLower = urlopen(href)
        soupLower = BeautifulSoup(pageLower, "html.parser")
        samples = soupLower.find_all('div', class_ = "sampleEntryBox")
        i=0
        for sample in samples:
            i += 1
            artistDetails = sample.find('div', class_="sampleTrackArtists").text if sample.find('div', class_="sampleTrackArtists") is not None else "" 
            albumDetails = sample.p.a.text if sample.p.a is not None else ""
            recordLabel = sample.find('span', itemprop="recordLabel").text if sample.find('span', itemprop="recordLabel") is not None else ""
            datePublished = sample.find('span', itemprop="datePublished").text if sample.find('span', itemprop="datePublished") is not None else ""
            producers = sample.find('div',class_="track-metainfo").text if sample.find('div',class_="track-metainfo") is not None else ""
            print(i, producers)

My Name Is
This track sampled  :  https://www.whosampled.com/sample/31/Eminem-My-Name-Is-Labi-Siffre-I-Got-The.../
1 
Producer:
                
                    Dr. Dre

2 
Producers:
                
                    Jim Sullivan,
                
                    Derek Lawrence

This track was sampled in  :  https://www.whosampled.com/sample/22671/Eminem-The-Real-Slim-Shady-Eminem-My-Name-Is/
1 
Producers:
                
                    Dr. Dre,
                
                    Mel-Man

2 
Producer:
                
                    Dr. Dre

What's the Difference
This track sampled  :  https://www.whosampled.com/sample/147/Dr.-Dre-Eminem-Xzibit-What%27s-the-Difference-Charles-Aznavour-Parce-Que-Tu-Crois/
1 
Producers:
                
                    Dr. Dre,
                
                    Mel-Man

2 
This track was sampled in  :  https://www.whosampled.com/sample/146/Blu-Cantrell-Sean-Paul-Breathe-Dr.-Dre-Eminem-Xzibit-What%27s-the-Difference/
1 
Pro

In [174]:
artistList = []
for pageNumber in range(1):
    url = "https://www.whosampled.com/" + artist + "/?sp=" + str(pageNumber+1)
    print(url)
    page = urlopen(url)
    soup = BeautifulSoup(page, "html.parser")
    
    tracks = soup.find_all('article', class_ = 'trackItem')
    #print(type(tracks))
    #print(len(tracks))

    for track in tracks:
        # Get the details of the current song on the list
        trackName = track.h3.a.text if track.h3.a is not None else ""
        trackRef = "https://www.whosampled.com" + track.h3.a['href']
        trackReleaseYear = track.h3.find('span', class_="trackYear").text if track.h3.find('span', class_="trackYear") is not None else ""
        #print(trackName)

        # Get the details of the sampled songs
        track_connections = track.find_all('div', class_ = "track-connection")
        for track_connection in track_connections:
            sample = track_connection.span.text.strip()
            href = "https://www.whosampled.com" + track_connection.ul.li.a['href']
            #print("This track", sample, " : ", href)
            pageLower = urlopen(href)
            soupLower = BeautifulSoup(pageLower, "html.parser")
            samples = soupLower.find_all('div', class_ = "sampleEntryBox")
            
            for sample in samples:
                artistDetails = sample.find('div', class_="sampleTrackArtists").text if sample.find('div', class_="sampleTrackArtists") is not None else "" 
                albumDetails = sample.p.a.text if sample.p.a is not None else ""
                recordLabel = sample.find('span', itemprop="recordLabel").text if sample.find('span', itemprop="recordLabel") is not None else ""
                datePublished = sample.find('span', itemprop="datePublished").text if sample.find('span', itemprop="datePublished") is not None else ""
                producers = sample.find('div',class_="track-metainfo").text if sample.find('div',class_="track-metainfo") is not None else ""
                
                # Start adding the data into a dataframe
                artistList.append([artist, artistDetails, trackName, recordLabel,trackReleaseYear, producers, artistDetails])
#print(artistList)
df = pd.DataFrame(artistList, columns = ['Main Artist','Main Featured Artist','Main Song Title', 'Main Record Label','Main Album Year', 
                                        'Main Producers', 'Sampled Artist'])
print(df)

https://www.whosampled.com/Eminem/?sp=1
   Main Artist                           Main Featured Artist  \
0       Eminem                                 \nEminem\n\n\n   
1       Eminem                            \nLabi Siffre\n\n\n   
2       Eminem                                 \nEminem\n\n\n   
3       Eminem                                 \nEminem\n\n\n   
4       Eminem        \nDr. Dre feat. Eminem and Xzibit\n\n\n   
5       Eminem                       \nCharles Aznavour\n\n\n   
6       Eminem           \nBlu Cantrell feat. Sean Paul\n\n\n   
7       Eminem        \nDr. Dre feat. Eminem and Xzibit\n\n\n   
8       Eminem                  \nBig Sean feat. Eminem\n\n\n   
9       Eminem     \nQuincy Jones feat. Valerie Simpson\n\n\n   
10      Eminem            \nEminem, Dr. Dre and 50 Cent\n\n\n   
11      Eminem                             \nMike Brant\n\n\n   
12      Eminem  \nReef the Lost Cauze and King Magnetic\n\n\n   
13      Eminem            \nEminem, Dr. Dre and 50

In [270]:
artists = ["2pac","Biggie","Jay-Z","Nas","Drake","Kendrick-Lamar","Lil%27-Kim","Nicki-Minaj","Ice-Cube","Dr.-Dre","Snoop-Dogg","Queen-Latifah","Lauryn-Hill","Kanye-West","Eminem","Lil-Wayne","50-Cent","Busta-Rhymes","Common","Puff-Daddy","DMX","Rakim","KRS-One","LL-Cool-J","Queen-Latifah","MC-Lyte","Fugees","N.W.A","Public-Enemy","Run-DMC","Salt-N-Pepa","UGK","Outkast","Wu-Tang-Clan"]

overallList = []

for artist in artists:
    for pageNumber in range(100):
        url = "https://www.whosampled.com/" + artist + "/?sp=" + str(pageNumber+1)
        #print(url)
        page = urlopen(url)
        soup = BeautifulSoup(page, "html.parser")

        # Loop through each track and get their details
        tracks = soup.find_all('article', class_ = 'trackItem')

        for track in tracks:
            # Get the details of the current song on the list
            trackName = track.h3.a.text if track.h3.a is not None else ""
            #print("Currently working on track : ", trackName)
            #trackRef = "https://www.whosampled.com" + track.h3.a['href']
            #trackReleaseYear = track.h3.find('span', class_="trackYear").text if track.h3.find('span', class_="trackYear") is not None else ""
            #print(trackName)

            # Get the details of the sampled songs
            track_connections = track.find_all('div', class_ = "track-connection")
            for track_connection in track_connections:
                sample = track_connection.span.text.strip()
                href = "https://www.whosampled.com" + track_connection.ul.li.a['href']
                #print("This track", sample, " : ", href)
                pageLower = urlopen(href)
                soupLower = BeautifulSoup(pageLower, "html.parser")

                # Destination song
                destinationSong = soupLower.find('div', id = "sampleWrap_dest")
                destinationSongHref = "https://www.whosampled.com" + destinationSong.h3.find('a', itemprop = 'url')['href']
                #print('destination', destinationSongHref)
                destinationURL = urlopen(destinationSongHref)
                destination = BeautifulSoup(destinationURL, "html.parser")

                # Collect all the main song information from here
                mainSongName = destination.find('h1', itemprop='name').text if destination.find('h1', itemprop='name') is not None else ""
                #print(mainSongName)
                mainArtistsName = destination.find('h2', class_='h2-heading').text if destination.find('h2', class_='h2-heading') is not None else ""
                #print(mainArtistsName)
                mainAlbumName = destination.find('h3', class_='release-name').a.text if destination.find('h3', class_='release-name').a is not None else ""
                mainAlbumYear = destination.find('span', itemprop="datePublished").text if destination.find('span', itemprop="datePublished") is not None else ""
                mainAlbumRecordLabel = destination.find('span', itemprop="recordLabel").text if destination.find('span', itemprop="recordLabel") is not None else ""
                mainProducer = destination.find('div',class_="track-metainfo").h3.text if destination.find('div',class_="track-metainfo").h3 is not None else ""
                mainGenre = destination.find('span', itemprop = 'genre').text if destination.find('span', itemprop = 'genre') is not None else ""

                mainProducers = mainProducer.replace("\n","").split(",")
                mainProducer = [mainProducer.strip() for mainProducer in mainProducers]

                mainGenre = mainGenre.replace("/",",")

                mainArtistsName = mainArtistsName.split("feat.")

                if(len(mainArtistsName)) > 1:
                    mainArtistName = mainArtistsName[0]
                    featuredArtistName = mainArtistsName[1].strip()
                    #print(mainArtistName)
                    #print(featuredArtistName)
                else:
                    mainArtistName = mainArtistsName[0]
                    featuredArtistName = ""
                #print(mainAlbumName)
                #print(mainAlbumYear)
                #print(mainAlbumRecordLabel)
                #print(mainProducer)
                #print(mainGenre)

                # Source song
                sourceSong = soupLower.find('div', id = "sampleWrap_source")
                sourceSongHref = "https://www.whosampled.com" + sourceSong.h3.find('a', itemprop='url')['href']
                #print('source',sourceSongHref)
                sourceURL = urlopen(sourceSongHref)
                source = BeautifulSoup(sourceURL, "html.parser")

                # Collect all the sampled song information from here
                sampledSongName = source.find('h1', itemprop='name').text if source.find('h1', itemprop='name') is not None else ""
                #print(mainSongName)
                sampledArtistsName = source.find('h2', class_='h2-heading').text if source.find('h2', class_='h2-heading') is not None else ""
                #print(mainArtistsName)
                sampledAlbumName = source.find('h3', class_='release-name').a.text if source.find('h3', class_='release-name').a is not None else ""
                sampledAlbumYear = source.find('span', itemprop="datePublished").text if source.find('span', itemprop="datePublished") is not None else ""
                sampledAlbumRecordLabel = source.find('span', itemprop="recordLabel").text if source.find('span', itemprop="recordLabel") is not None else ""
                sampledProducer = source.find('div',class_="track-metainfo").h3.text if source.find('div',class_="track-metainfo").h3 is not None else ""
                sampledGenre = source.find('span', itemprop = 'genre').text if source.find('span', itemprop = 'genre') is not None else ""
                sampledProducers = sampledProducer.replace("\n","").split(",")
                sampledProducer = [sampledProducer.strip() for sampledProducer in sampledProducers]
                sampledGenre = sampledGenre.replace("/",",")
                #print(sampledAlbumName)
                #print(sampledAlbumYear)
                #print(sampledAlbumRecordLabel)
                #print(sampledProducer)
                #print(sampledGenre)

                sampledArtistsName = sampledArtistsName.split("feat.")

                if(len(sampledArtistsName)) > 1:
                    sampledArtistName = sampledArtistsName[0]
                    featuredSampledArtistName = sampledArtistsName[1].strip()
                    #print(mainArtistName)
                    #print(featuredArtistName)
                else:
                    sampledArtistName = sampledArtistsName[0]
                    featuredSampledArtistName = ""


                overallList.append([mainArtistName, featuredArtistName, mainSongName,  mainAlbumName, mainAlbumYear, mainAlbumRecordLabel, ','.join(mainProducer), mainGenre, sampledArtistName, featuredSampledArtistName, sampledSongName, sampledAlbumName, sampledAlbumYear, sampledAlbumRecordLabel, ','.join(sampledProducer), sampledGenre])
            
#print(overallList)
df = pd.DataFrame(overallList, columns = ('Main Artist Name', "Main Featured Artist Name", 'Main Song Name', 'Main Album Name', 'Main Album Year', 'Main Album Record Label', 'Main Producers', 'Main Genre', 'Sampled Artist Name', 'Sampled Featured Astist Name', 'Sampled Song Name', 'Sampled Album Name', 'Sampled Album Year', 'Sampled Album Record Label', 'Sampled Producers', 'Sampled Genre'))


KeyboardInterrupt: 

In [275]:
artists = ["2pac"]

overallList = []

for artist in artists:
    for pageNumber in range(1):
        url = "https://www.whosampled.com/" + artist + "/?sp=" + str(pageNumber+1)
        #print(url)
        page = urlopen(url)
        soup = BeautifulSoup(page, "html.parser")

        # Loop through each track and get their details
        tracks = soup.find_all('article', class_ = 'trackItem')

        for track in tracks:
            # Get the details of the current song on the list
            trackName = track.h3.a.text if track.h3.a is not None else ""
            #print("Currently working on track : ", trackName)
            #trackRef = "https://www.whosampled.com" + track.h3.a['href']
            #trackReleaseYear = track.h3.find('span', class_="trackYear").text if track.h3.find('span', class_="trackYear") is not None else ""
            #print(trackName)

            # Get the details of the sampled songs
            track_connections = track.find_all('div', class_ = "track-connection")
            for track_connection in track_connections:
                #sample = track_connection.span.text.strip()
                samplesd = track_connection.ul.find_all('li')
                i = 0
                for sample in samplesd:
                    i+=1
                    print("i",i ,"sample",sample.find('a', class_="connectionName")['href'])
                

i 1 sample /sample/176/2Pac-Dr.-Dre-Roger-Troutman-California-Love-Joe-Cocker-Woman-to-Woman/
i 2 sample /sample/474/2Pac-Dr.-Dre-Roger-Troutman-California-Love-Zapp-Dance-Floor/
i 3 sample /sample/3027/2Pac-Dr.-Dre-Roger-Troutman-California-Love-Ronnie-Hudson-%26-The-Street-People-West-Coast-Poplock/
i 1 sample /sample/733/2Pac-Outlawz-Hit-%27Em-Up-Dennis-Edwards-Siedah-Garrett-Don%27t-Look-Any-Further/
i 2 sample /sample/6049/2Pac-Outlawz-Hit-%27Em-Up-Junior-M.A.F.I.A.-The-Notorious-B.I.G.-Player%27s-Anthem/
i 3 sample /sample/33431/2Pac-Outlawz-Hit-%27Em-Up-MC-Lyte-Audio-Two-10%25-Dis/
i 1 sample /sample/355/2Pac-Dear-Mama-Joe-Sample-In-All-My-Wildest-Dreams/
i 2 sample /sample/12823/2Pac-Dear-Mama-The-Spinners-Sadie/
i 1 sample /sample/211225/Tyga-Chris-Brown-For-the-Road-2Pac-Dear-Mama/
i 1 sample /sample/356/2Pac-Eric-Williams-Do-for-Love-Bobby-Caldwell-What-You-Won%27t-Do-for-Love/
i 2 sample /sample/69477/2Pac-Eric-Williams-Do-for-Love-The-Pharcyde-Y%3F-(Be-Like-That)-(Jay-Dee-

In [296]:
artists = ["2pac"]#,"Biggie","Jay-Z","Nas","Drake","Kendrick-Lamar","Lil%27-Kim","Nicki-Minaj","Ice-Cube","Dr.-Dre","Snoop-Dogg","Queen-Latifah","Lauryn-Hill","Kanye-West","Eminem","Lil-Wayne","50-Cent","Busta-Rhymes","Common","Puff-Daddy","DMX","Rakim","KRS-One","LL-Cool-J","Queen-Latifah","MC-Lyte","Fugees","N.W.A","Public-Enemy","Run-DMC","Salt-N-Pepa","UGK","Outkast","Wu-Tang-Clan"]

overallList = []

for artist in artists:
    print("Starting artist : ", artist)
    for pageNumber in range(1):
        url = "https://www.whosampled.com/" + artist + "/?sp=" + str(pageNumber+1)
        #print(url)
        page = urlopen(url)
        soup = BeautifulSoup(page, "html.parser")

        # Loop through each track and get their details
        tracks = soup.find_all('article', class_ = 'trackItem')

        for track in tracks:
            # Get the details of the current song on the list
            #trackName = track.h3.a.text if track.h3.a is not None else ""
            #print("Currently working on track : ", trackName)
            #trackRef = "https://www.whosampled.com" + track.h3.a['href']
            #trackReleaseYear = track.h3.find('span', class_="trackYear").text if track.h3.find('span', class_="trackYear") is not None else ""
            #print(trackName)
                
            # Get the details of the sampled songs
            track_connections = track.find_all('div', class_ = "track-connection")
            for track_connection in track_connections:
                samplesd = track_connection.ul.find_all('li')
                #sample = track_connection.span.text.strip()
                for sample in samplesd:
                    href = "https://www.whosampled.com" + sample.find('a', class_="connectionName")['href']
                    #print("This track", sample, " : ", href)
                    pageLower = urlopen(href)
                    soupLower = BeautifulSoup(pageLower, "html.parser")

                    # Destination song
                    destinationSong = soupLower.find('div', id = "sampleWrap_dest")
                    destinationSongHref = "https://www.whosampled.com" + destinationSong.h3.find('a', itemprop = 'url')['href']
                    #print('destination', destinationSongHref)
                    destinationURL = urlopen(destinationSongHref)
                    destination = BeautifulSoup(destinationURL, "html.parser")

                    # Collect all the main song information from here
                    mainSongName = destination.find('h1', itemprop='name').text if destination.find('h1', itemprop='name') is not None else ""
                    #print(mainSongName)
                    mainArtistsName = destination.find('h2', class_='h2-heading').text if destination.find('h2', class_='h2-heading') is not None else ""
                    #print(mainArtistsName)
                    mainAlbumName = destination.find('h3', class_='release-name').a.text if destination.find('h3', class_='release-name') is not None else ""
                    mainAlbumYear = destination.find('span', itemprop="datePublished").text if destination.find('span', itemprop="datePublished") is not None else ""
                    mainAlbumRecordLabel = destination.find('span', itemprop="recordLabel").text if destination.find('span', itemprop="recordLabel") is not None else ""
                    
                    mainProducers = destination.find('div',class_="track-metainfo").find_all('span', itemprop='name') if destination.find('div',class_="track-metainfo").find_all('span', itemprop='name') is not None else ""
                    print([mainProducer.text for mainProducer in mainProducers])
                    
                   

Starting artist :  2pac
['Dr. Dre']
['Dr. Dre']
['Dr. Dre']
['Johnny J']
['Johnny J']
['Johnny J']
['Tony Pizarro']
['Tony Pizarro']
['Lil C', 'Lamar "Marz" Edwards', 'Jess Jackson']
['Soulshock & Karlin']
['Soulshock & Karlin']
['Ducko Mcfli']
['2Pac']
['2Pac']
['2Pac']
['Johnny J']
['The Monarch', 'DJ Khaled']
['FreeStyleGames']
['Stretch']
['Stretch']
['Elite', 'J. Cole']
['Daz Dillinger']
[]
[]
['DJ Daryl']
['DJ Daryl']
['DJ Daryl']
['D-Flow Production Squad']
['D-Flow Production Squad']
['D-Flow Production Squad']


In [287]:
df

,Main Artist Name,Main Featured Artist Name,Main Song Name,Main Album Name,Main Album Year,Main Album Record Label,Main Producers,Main Genre,Sampled Artist Name,Sampled Featured Astist Name,Sampled Song Name,Sampled Album Name,Sampled Album Year,Sampled Album Record Label,Sampled Producers,Sampled Genre
0,2Pac,Dr. Dre and Roger Troutman,California Love,All Eyez on Me,1995,Death Row,Producer: Dr. Dre,"Hip-Hop , Rap , R&B",Joe Cocker,,Woman to Woman,Joe Cocker,1972,A&M,Producer: Nigel Thomas,"Soul , Funk , Disco"
1,2Pac,Dr. Dre and Roger Troutman,California Love,All Eyez on Me,1995,Death Row,Producer: Dr. Dre,"Hip-Hop , Rap , R&B",Zapp,,Dance Floor,Zapp II,1982,Warner Bros.,Producers: Roger Troutm...,"Soul , Funk , Disco"
2,2Pac,Dr. Dre and Roger Troutman,California Love,All Eyez on Me,1995,Death Row,Producer: Dr. Dre,"Hip-Hop , Rap , R&B",Ronnie Hudson & The Street People,,West Coast Poplock,,1982,Street People,Producer: Mikel Hooks,"Soul , Funk , Disco"
3,2Pac,Outlawz,Hit 'Em Up,How Do You Want It,1996,Death Row,Producer: Johnny J,"Hip-Hop , Rap , R&B",Dennis Edwards,Siedah Garrett,Don't Look Any Further,Don't Look Any Further,1984,Gordy,Producer: Dennis Edwards,"Hip-Hop , Rap , R&B"
4,2Pac,Outlawz,Hit 'Em Up,How Do You Want It,1996,Death Row,Producer: Johnny J,"Hip-Hop , Rap , R&B",Junior M.A.F.I.A.,The Notorious B.I.G.,Player's Anthem,Conspiracy,1995,Big Beat,Producer: Clark Kent,"Hip-Hop , Rap , R&B"
5,2Pac,Outlawz,Hit 'Em Up,How Do You Want It,1996,Death Row,Producer: Johnny J,"Hip-Hop , Rap , R&B",MC Lyte,Audio Two,10% Dis,Lyte as a Rock,1988,First Priority,Producer: Audio Two,"Hip-Hop , Rap , R&B"
6,2Pac,,Dear Mama,Me Against the World,1995,Interscope,Producer: Tony Pizarro,"Hip-Hop , Rap , R&B",Joe Sample,,In All My Wildest Dreams,Rainbow Seeker,1978,ABC,Producer: The Crusaders,"Jazz , Blues"
7,2Pac,,Dear Mama,Me Against the World,1995,Interscope,Producer: Tony Pizarro,"Hip-Hop , Rap , R&B",The Spinners,,Sadie,New and Improved,1974,Atlantic,Producer: Thom Bell,"Soul , Funk , Disco"
8,Tyga,Chris Brown,For the Road,Hotel California,2013,Cash Money,"Producers: Lil C,Lamar ...","Hip-Hop , Rap , R&B",2Pac,,Dear Mama,Me Against the World,1995,Interscope,Producer: Tony Pizarro,"Hip-Hop , Rap , R&B"
9,2Pac,Eric Williams,Do for Love,R U Still Down? (Remember Me),1997,Interscope,Producer: Soulshock & K...,"Hip-Hop , Rap , R&B",Bobby Caldwell,,What You Won't Do for Love,Bobby Caldwell,1978,Clouds,Producer: Ann Holloway,"Soul , Funk , Disco"


In [392]:
artists = ["The-Notorious-B.I.G."] #"2pac","Jay-Z","Nas","Drake","Kendrick-Lamar","Lil%27-Kim","Nicki-Minaj","Ice-Cube","Dr.-Dre","Snoop-Dogg","Queen-Latifah","Lauryn-Hill","Kanye-West","Eminem","Lil-Wayne","50-Cent","Busta-Rhymes","Common","Puff-Daddy","DMX","Rakim","KRS-One","LL-Cool-J","Queen-Latifah","MC-Lyte","Fugees","N.W.A","Public-Enemy","Run-DMC","Salt-N-Pepa","UGK","Outkast","Wu-Tang-Clan"]
artDict = {}
for artist in artists:
    url = "https://www.whosampled.com/" + artist.strip() + "/?sp="
    page = urlopen(url)
    soup = BeautifulSoup(page, "html.parser")
    pageNumberList = soup.find_all('span', class_="page")
    pageNumber = []
    for pageNumbers in pageNumberList:
        pageNumber.append(pageNumbers.find('a').text)
        
        #artDict[artist] = url
    #print(pageNumber[len(pageNumber)-1])
    artDict[artist] = [url, pageNumber]

for artist, url in artDict.items():
    print(artist, url[0], url[1])
#artist, url = artDict.items()
#print(artist)

The-Notorious-B.I.G. https://www.whosampled.com/The-Notorious-B.I.G./?sp= ['2', '3', '4', '5', '6', '7', '8', '21']


In [8]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

def artistURLMake(artists, artDict):
    for artist in artists:
        print("Starting artist : ", artist)
        url = "https://www.whosampled.com/" + artist.strip() + "/?sp="
        page = urlopen(url)
        soup = BeautifulSoup(page, "html.parser")
        pageNumberList = soup.find_all('span', class_="page")
        pageNumber = []
        for pageNumbers in pageNumberList:
            pageNumber.append(pageNumbers.find('a').text)

            #artDict[artist] = url
        #print(pageNumber[len(pageNumber)-1])
        artDict[artist] = [url, pageNumber]
    return artDict     

def scrapeData(artDict):
    print("Starting scrape data function")
    for artist, artistURLDetails in artDict.items():
        print("Starting artist : ", artist)
        #for artistURL in artistURLs:
        pageNumbers = artistURLDetails[1]
        #for artist in artists:
        overallList = []    
        for pageNumber in range(int(pageNumbers[len(pageNumbers)-1])):
        #for pageNumber in range(1):
            #url = "https://www.whosampled.com/" + artist + "/?sp=" + str(pageNumber+1)
            #url = str(url)
            url = artistURLDetails[0] + str(pageNumber+1)
            #print("Working with url : ", url)
            page = urlopen(url)
            soup = BeautifulSoup(page, "html.parser")

            # Loop through each track and get their details
            tracks = soup.find_all('article', class_ = 'trackItem')

            for track in tracks:
                # Get the details of the current song on the list
                #trackName = track.h3.a.text if track.h3.a is not None else ""
                #print("Currently working on track : ", trackName)
                #trackRef = "https://www.whosampled.com" + track.h3.a['href']
                #trackReleaseYear = track.h3.find('span', class_="trackYear").text if track.h3.find('span', class_="trackYear") is not None else ""
                #print(trackName)

                # Get the details of the sampled songs
                track_connections = track.find_all('div', class_ = "track-connection")
                for track_connection in track_connections:
                    samplesd = track_connection.ul.find_all('li')
                    #sample = track_connection.span.text.strip()
                    for sample in samplesd:
                        href = "https://www.whosampled.com" + sample.find('a', class_="connectionName")['href']
                        #print("This track", sample, " : ", href)
                        pageLower = urlopen(href)
                        soupLower = BeautifulSoup(pageLower, "html.parser")

                        # Destination song
                        destinationSong = soupLower.find('div', id = "sampleWrap_dest")
                        destinationSongHref = "https://www.whosampled.com" + destinationSong.h3.find('a', itemprop = 'url')['href'] if destinationSong.h3 is not None else ""
                        #print('destination', destinationSongHref)
                        if destinationSongHref != '':
                            destinationURL = urlopen(destinationSongHref)
                        else:
                            continue            
                        
                        destination = BeautifulSoup(destinationURL, "html.parser")

                        # Collect all the main song information from here
                        mainSongName = destination.find('h1', itemprop='name').text if destination.find('h1', itemprop='name') is not None else ""
                        #print(mainSongName)
                        mainArtistsName = destination.find('h2', class_='h2-heading').text if destination.find('h2', class_='h2-heading') is not None else ""
                        #print(mainArtistsName)
                        mainAlbumName = destination.find('h3', class_='release-name').a.text if destination.find('h3', class_='release-name') is not None else ""
                        mainAlbumYear = destination.find('span', itemprop="datePublished").text if destination.find('span', itemprop="datePublished") is not None else ""
                        mainAlbumRecordLabel = destination.find('span', itemprop="recordLabel").text if destination.find('span', itemprop="recordLabel") is not None else ""

                        mainProducers = destination.find('div',class_="track-metainfo").find_all('span', itemprop='name') if destination.find('div',class_="track-metainfo").find_all('span', itemprop='name') is not None else ""
                        #print(mainProducer)

                        mainGenre = destination.find('span', itemprop = 'genre').text if destination.find('span', itemprop = 'genre') is not None else ""

                        #mainProducers = mainProducer.replace("\n","").split(",")
                        mainProducer = [mainProds.text for mainProds in mainProducers]

                        mainGenre = mainGenre.replace("/",",")

                        mainArtistsName = mainArtistsName.split("feat.")

                        if(len(mainArtistsName)) > 1:
                            mainArtistName = mainArtistsName[0]
                            featuredArtistName = mainArtistsName[1].strip()
                            #print(mainArtistName)
                            #print(featuredArtistName)
                        else:
                            mainArtistName = mainArtistsName[0]
                            featuredArtistName = ""
                        #print(mainAlbumName)
                        #print(mainAlbumYear)
                        #print(mainAlbumRecordLabel)
                        #print(mainProducer)
                        #print(mainGenre)

                        # Source song
                        sourceSong = soupLower.find('div', id = "sampleWrap_source")
                        sourceSongHref = "https://www.whosampled.com" + sourceSong.h3.find('a', itemprop='url')['href'] if sourceSong.h3 is not None else ""
                        #print('source',sourceSongHref)
                        if sourceSongHref != '':
                            sourceURL = urlopen(sourceSongHref)
                        else:
                            continue
                        source = BeautifulSoup(sourceURL, "html.parser")

                        # Collect all the sampled song information from here
                        sampledSongName = source.find('h1', itemprop='name').text if source.find('h1', itemprop='name') is not None else ""
                        #print(mainSongName)
                        sampledArtistsName = source.find('h2', class_='h2-heading').text if source.find('h2', class_='h2-heading') is not None else ""
                        #print(mainArtistsName)
                        sampledAlbumName = source.find('h3', class_='release-name').a.text if source.find('h3', class_='release-name') is not None else ""
                        sampledAlbumYear = source.find('span', itemprop="datePublished").text if source.find('span', itemprop="datePublished") is not None else ""
                        sampledAlbumRecordLabel = source.find('span', itemprop="recordLabel").text if source.find('span', itemprop="recordLabel") is not None else ""

                        sampledProducers = source.find('div',class_="track-metainfo").find_all('span', itemprop='name') if source.find('div',class_="track-metainfo").find_all('span', itemprop='name') is not None else ""

                        sampledGenre = source.find('span', itemprop = 'genre').text if source.find('span', itemprop = 'genre') is not None else ""

                        #sampledProducers = sampledProducer.replace("\n","").split(",")

                        sampledProducer = [sampledProd.text for sampledProd in sampledProducers]

                        sampledGenre = sampledGenre.replace("/",",")
                        #print(sampledAlbumName)
                        #print(sampledAlbumYear)
                        #print(sampledAlbumRecordLabel)
                        #print(sampledProducer)
                        #print(sampledGenre)

                        sampledArtistsName = sampledArtistsName.split("feat.")

                        if(len(sampledArtistsName)) > 1:
                            sampledArtistName = sampledArtistsName[0]
                            featuredSampledArtistName = sampledArtistsName[1].strip()
                            #print(mainArtistName)
                            #print(featuredArtistName)
                        else:
                            sampledArtistName = sampledArtistsName[0]
                            featuredSampledArtistName = ""


                        overallList.append([url, mainArtistName, featuredArtistName, mainSongName,  mainAlbumName, mainAlbumYear, mainAlbumRecordLabel, ','.join(mainProducer), mainGenre, sampledArtistName, featuredSampledArtistName, sampledSongName, sampledAlbumName, sampledAlbumYear, sampledAlbumRecordLabel, ','.join(sampledProducer), sampledGenre])

        #print(overallList)
        df = pd.DataFrame(overallList, columns = ('Link Page','Main Artist Name', "Main Featured Artist Name", 'Main Song Name', 'Main Album Name', 'Main Album Year', 'Main Album Record Label', 'Main Producers', 'Main Genre', 'Sampled Artist Name', 'Sampled Featured Astist Name', 'Sampled Song Name', 'Sampled Album Name', 'Sampled Album Year', 'Sampled Album Record Label', 'Sampled Producers', 'Sampled Genre'))

        # Dump the results to csv file
        df.to_csv(artist + '.csv')
 

In [9]:
#from multiprocessing import Pool
artDict = {}
artists = ["Jay-Z","Nas","Drake","Kendrick-Lamar","Lil%27-Kim","Nicki-Minaj","Ice-Cube","Dr.-Dre","Snoop-Dogg","Queen-Latifah","Lauryn-Hill","Kanye-West","Eminem","Lil-Wayne","50-Cent","Busta-Rhymes","Common","Puff-Daddy","DMX","Rakim","KRS-One","LL-Cool-J","Queen-Latifah","MC-Lyte","Fugees","N.W.A","Public-Enemy","Run-DMC","Salt-N-Pepa","UGK","Outkast","Wu-Tang-Clan"] 
artDict = artistURLMake(artists, artDict)
#print(artDict)

scrapeData(artDict)
#if __name__ == '__main__':
#    with Pool(10) as p:
#        p.map(scrapeData,artDict)
        #records = p.map(parse, cars_links)
    
    


Starting artist :  Jay-Z
Starting artist :  Nas
Starting artist :  Drake
Starting artist :  Kendrick-Lamar
Starting artist :  Lil%27-Kim
Starting artist :  Nicki-Minaj
Starting artist :  Ice-Cube
Starting artist :  Dr.-Dre
Starting artist :  Snoop-Dogg
Starting artist :  Queen-Latifah
Starting artist :  Lauryn-Hill
Starting artist :  Kanye-West
Starting artist :  Eminem
Starting artist :  Lil-Wayne
Starting artist :  50-Cent
Starting artist :  Busta-Rhymes
Starting artist :  Common
Starting artist :  Puff-Daddy
Starting artist :  DMX
Starting artist :  Rakim
Starting artist :  KRS-One
Starting artist :  LL-Cool-J
Starting artist :  Queen-Latifah
Starting artist :  MC-Lyte
Starting artist :  Fugees
Starting artist :  N.W.A
Starting artist :  Public-Enemy
Starting artist :  Run-DMC
Starting artist :  Salt-N-Pepa
Starting artist :  UGK
Starting artist :  Outkast
Starting artist :  Wu-Tang-Clan
Starting scrape data function
Starting artist :  Jay-Z
Starting artist :  Nas
Starting artist : 

In [333]:
artists = ["2pac","The-Notorious-B.I.G.","Jay-Z","Nas","Drake","Kendrick-Lamar","Lil%27-Kim","Nicki-Minaj","Ice-Cube","Dr.-Dre","Snoop-Dogg","Queen-Latifah","Lauryn-Hill","Kanye-West","Eminem","Lil-Wayne","50-Cent","Busta-Rhymes","Common","Puff-Daddy","DMX","Rakim","KRS-One","LL-Cool-J","Queen-Latifah","MC-Lyte","Fugees","N.W.A","Public-Enemy","Run-DMC","Salt-N-Pepa","UGK","Outkast","Wu-Tang-Clan"]
artDict = {}
for artist in artists:
    artistURLList = []
    for pageNumber in range(100):
        url = "https://www.whosampled.com/" + artist + "/?sp=" + str(pageNumber+1)
        artistURLList.append(url)
    artDict[artist] = artistURLList
print(artDict)

{'2pac': ['https://www.whosampled.com/2pac/?sp=1', 'https://www.whosampled.com/2pac/?sp=2', 'https://www.whosampled.com/2pac/?sp=3', 'https://www.whosampled.com/2pac/?sp=4', 'https://www.whosampled.com/2pac/?sp=5', 'https://www.whosampled.com/2pac/?sp=6', 'https://www.whosampled.com/2pac/?sp=7', 'https://www.whosampled.com/2pac/?sp=8', 'https://www.whosampled.com/2pac/?sp=9', 'https://www.whosampled.com/2pac/?sp=10', 'https://www.whosampled.com/2pac/?sp=11', 'https://www.whosampled.com/2pac/?sp=12', 'https://www.whosampled.com/2pac/?sp=13', 'https://www.whosampled.com/2pac/?sp=14', 'https://www.whosampled.com/2pac/?sp=15', 'https://www.whosampled.com/2pac/?sp=16', 'https://www.whosampled.com/2pac/?sp=17', 'https://www.whosampled.com/2pac/?sp=18', 'https://www.whosampled.com/2pac/?sp=19', 'https://www.whosampled.com/2pac/?sp=20', 'https://www.whosampled.com/2pac/?sp=21', 'https://www.whosampled.com/2pac/?sp=22', 'https://www.whosampled.com/2pac/?sp=23', 'https://www.whosampled.com/2pac/

In [329]:
import requests
request = requests.get('https://www.whosampled.com/2Pac/?sp=4')
if request.status_code == 200:
    print('Web site exists')
else:
    print('Web site does not exist') 

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [330]:
import httplib
c = httplib.HTTPConnection('https://www.whosampled.com/2Pac/?sp=4')
c.request("HEAD", '')
if c.getresponse().status == 200:
   print('web site exists')

ModuleNotFoundError: No module named 'httplib'

In [338]:
artists = ["2pac"]#,"The-Notorious-B.I.G.","Jay-Z","Nas","Drake","Kendrick-Lamar","Lil%27-Kim","Nicki-Minaj","Ice-Cube","Dr.-Dre","Snoop-Dogg","Queen-Latifah","Lauryn-Hill","Kanye-West","Eminem","Lil-Wayne","50-Cent","Busta-Rhymes","Common","Puff-Daddy","DMX","Rakim","KRS-One","LL-Cool-J","Queen-Latifah","MC-Lyte","Fugees","N.W.A","Public-Enemy","Run-DMC","Salt-N-Pepa","UGK","Outkast","Wu-Tang-Clan"]
artDict = {}
for artist in artists:
    artistURLList = []
    for pageNumber in range(2):
        url = "https://www.whosampled.com/" + artist.strip() + "/?sp="
        artistURLList.append(url)
    artDict[artist] = artistURLList
    
print(artDict)

{'2pac': ['https://www.whosampled.com/2pac/?sp=', 'https://www.whosampled.com/2pac/?sp=']}


In [340]:
artists = ["2pac"]#,"The-Notorious-B.I.G.","Jay-Z","Nas","Drake","Kendrick-Lamar","Lil%27-Kim","Nicki-Minaj","Ice-Cube","Dr.-Dre","Snoop-Dogg","Queen-Latifah","Lauryn-Hill","Kanye-West","Eminem","Lil-Wayne","50-Cent","Busta-Rhymes","Common","Puff-Daddy","DMX","Rakim","KRS-One","LL-Cool-J","Queen-Latifah","MC-Lyte","Fugees","N.W.A","Public-Enemy","Run-DMC","Salt-N-Pepa","UGK","Outkast","Wu-Tang-Clan"]
artDict = {}
for artist in artists:
    url = "https://www.whosampled.com/" + artist.strip() + "/?sp="
    artDict[artist] = url
    
print(artDict)

{'2pac': 'https://www.whosampled.com/2pac/?sp='}


In [355]:
import urllib
urllib.urlencode()

AttributeError: module 'urllib' has no attribute 'urlencode'

In [ ]:
import multiprocessing
import requests
def check_links(links):
    returnList = []
    print(links)
    r = requests.get(links)
    rc = r.status_code
    strRc = str(rc)
    result = links+' '+strRc
    return(result)

def main():
    pool = multiprocessing.Pool(4)
    a = ['http://stackoverflow.com','https://www.google.co.in']
    results = pool.map(check_links, a)

    pool.close()
    pool.join()

    for result in results:
        print(result)

if __name__ == '__main__':
    main()

In [2]:
def cube(x):
    return x**3

In [ ]:
import multiprocessing as mp
import random
import string
pool = mp.Pool(processes=4)
print(pool)
results = pool.map(cube, range(1,2))
print(results)

In [ ]:
Starting artist :  Outkast
